In [1]:
import requests
import os
import json
import TwitterSecrets


bearer_token = TwitterSecrets.Bearer_Token

In [2]:
user_id = 1009984504247676928 

search_url = "https://api.twitter.com/2/tweets/search/recent"
follower_url = f"https://api.twitter.com/2/users/{user_id}/followers"
following_url = f"https://api.twitter.com/2/users/{user_id}/following"
user_url = "https://api.twitter.com/2/users"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': '#MarchMadness'}

In [3]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [4]:
CACHE_FILENAME = "cache.json"

def open_cache():
    ''' opens the cache file if it exists and loads the JSON into
    the CACHE dictionary.

    if the cache file doesn't exist, creates a new cache dictionary

    Parameters
    ----------
    None

    Returns
    -------
    The opened cache
    '''
    try:
        cache_file = open(CACHE_FILENAME, 'r')
        cache_contents = cache_file.read()
        cache_dict = json.loads(cache_contents)
        cache_file.close()
    except:
        cache_dict = {}
    return cache_dict

def save_cache(cache_dict):
    ''' saves the current state of the cache to disk
    Parameters
    ----------
    cache_dict: dict
        The dictionary to save
    Returns
    -------
    None
    '''
    dumped_json_cache = json.dumps(cache_dict)
    fw = open(CACHE_FILENAME,"w")
    fw.write(dumped_json_cache)
    fw.close() 

def construct_unique_key(baseurl, params):
    param_strings = []
    connector = '_'
    for k in params.keys():
        param_strings.append(f'{k}_{params[k]}')
    unique_key = baseurl + connector + connector.join(param_strings)
    return unique_key

def twitter_with_cache(baseurl, params):
    unique_key = construct_unique_key(baseurl, params)
    if unique_key in CACHE:
        return CACHE[unique_key]
    else:
        CACHE[unique_key] = connect_to_endpoint(baseurl, params) 
        save_cache(CACHE)
        return CACHE[unique_key]

CACHE = open_cache()

In [5]:
class User:
    def __init__(self, id, name, username):
        self.id = id
        self.name = name
        self.username = username
        self.connectedTo = []
        # self.followers = []
        # self.following = []
    
    def addNeighbor(self, nbr):
        '''
        nbr (User): another vertex connected to this vertix
        '''
        self.connectedTo.append(nbr)
    def getId(self):
        return self.id
    def getConnections(self):
        return self.connectedTo
    def getConnectionIds(self):
        ids = [x.id for x in self.connectedTo]
        return ids
    def __str__(self):
        return str(self.id) + ' is connected to ' + str([x.id for x in self.connectedTo])



In [12]:
# Test

V1 = User(1, 'aa', 'aa123')
V2 = User(2, 'bb', 'bb234')
V3 = User(3, 'cc', 'cc345')
V1.addNeighbor(V2)
V1.addNeighbor(V3)
V1.__str__()

print(V1.getConnectionIds())
print(V2.getConnectionIds())

[2, 3]
[]


In [6]:
class Graph:
    def __init__(self):
        self.vertList = {} 
        self.numVertices = 0

    def addVertex(self, key, name, username): ## avoid repetitive?
        self.numVertices = self.numVertices + 1
        newVertex = User(key, name, username)
        self.vertList[key] = newVertex
        return newVertex

    def getVertex(self,n):
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None

    def __contains__(self,n):
        return n in self.vertList

    def addEdge(self,Follower_key, Followed_key, follower_name='name', followed_name='name2', follower_username='username1', followed_username='username2'):
        '''
        V1 -> V2
        '''
        if Follower_key not in self.vertList.keys():
            self.addVertex(Follower_key, follower_name, follower_username)
            # print(f"create a new vertix: {V1_key}")
        if Followed_key not in self.vertList.keys():
            self.addVertex(Followed_key, followed_name, followed_username)
            # print(f"create a new vertix: {V2_key}")
        if self.vertList[Followed_key] not in self.vertList[Follower_key].getConnections(): # if the edge doesn't exist
            self.vertList[Follower_key].addNeighbor(self.vertList[Followed_key])

    def getVertices(self):
        return self.vertList.keys()

    def adjList(self):
        adjList = []
        for v_key in self.vertList.keys():
            adjList.append(self.vertList[v_key].getConnectionIds())
        return adjList


    def __iter__(self):
        return iter(self.vertList.values())

In [25]:
# Test
network_test = Graph()
network_test.addEdge('t1', 't2')
print(network_test.getVertices())
for key in network_test.vertList.keys():
    print(network_test.vertList[key].name)
    print(network_test.vertList[key].getConnectionIds())


dict_keys(['t1', 't2'])
name
['t2']
name2
[]


## Pseudo code
1. User enter a user's username
2. Find the user's id
3. Use the id to get the follower list
4. Connect the user to his/her followers (add edges in the network)
5. Find the followings of the followers
6. Connect the followers to their followings (add edges in the network)
7. Find and show the top 3 users who have the most followers in the network
- run through the connection list to count the user appear for the most times?


In [7]:
# Test
small_network = Graph()
Followed_key = user_id # user input username -> id
followed_name = 'target'
followed_username = 'targetuser123'

followers = twitter_with_cache(follower_url, {'max_results': 5})
for user in followers['data']:
    small_network.addEdge(user['id'], Followed_key, user['name'], followed_name, user['username'], followed_username)
    
    user_id = user['id']
    following_url = f"https://api.twitter.com/2/users/{user_id}/following"
    following = twitter_with_cache(following_url, {'max_results': 5})
    for f in following['data']:
        small_network.addEdge(user['id'], f['id'], user['name'], f['name'], user['username'], f['username'])
# print(small_network.getVertices())

for key in small_network.vertList.keys():
    print(small_network.vertList[key].name, small_network.vertList[key].getConnectionIds())
    # print(small_network.vertList[key].name, small_network.vertList[key].username)


leca [1009984504247676928, '1598053484259348485', '1428302512319197189', '1524991563696586752', '1395435978131525632', '1009984504247676928']
target []
423 []
gege []
liz []
WOOSUNG []
원위 (ONEWE) []
Dwie [1009984504247676928, '1382576846336790536', '1058234696008589312', '893085958119211011', '1120217350848958465', '1413307716517261313']
JUST B []
VANNER 배너 []
온앤오프 (ONF) Official []
TO1 []
TEMPEST(템페스트) []
cloudtmn [1009984504247676928, '1585878485591298048', '1301764681430786048', '1230512150923243520', '1120359217553010688', '1116403957834768384']
연울 []
여백 []
안녕하서호！ []
🅼🅾🆉 []
yuyutzu 御御子 []
X_Hongggggg [1009984504247676928, '1135806843773542401', '1009984504247676928', '1024525554395963392', '1024629552306380800', '1248459032320327683']
SuperM []
더로즈_The Rose []
Sana []
BAND LUCY | 밴드 루시 []
Candy Sugar Pop / Betsy [1009984504247676928, '1084494155403968512', '861579438244241411', '1009984504247676928', '1219213114895302656', '1031445660841603072']
TXT Charts ❤️‍🩹 []
A.C.E []
WOODZ(조승

In [11]:
print(len(small_network.vertList))

top_dic = {}
most = ('', 0)
for key in small_network.vertList.keys():
    for id in small_network.vertList[key].getConnectionIds():
        if id not in top_dic.keys():
            top_dic[id] = 1
        else:
            top_dic[id] += 1
        if id != user_id and top_dic[id] > most[1]: #exclude the target user
            most = (id, top_dic[id])
    # print(small_network.vertList[key].name, small_network.vertList[key].getConnectionIds())
print(f"most = {most}")

user = twitter_with_cache(user_url, {'ids': [most[0]]})
print(user)

29
most = (1009984504247676928, 5)
{'data': [{'id': '1009984504247676928', 'name': '원위 (ONEWE)', 'username': 'official_ONEWE'}]}


In [21]:
def main():
    # json_response = connect_to_endpoint(search_url, query_params)
    followers = twitter_with_cache(follower_url, {'max_results': 5})
    print("\nfollowers:")
    for user in followers['data']:
        print(f"{user['id']}: {user['name']}, ({user['username']})")
    
    ## Next page
    # next_token = json_response['meta']['next_token']
    # print(f"\nnext page token: {next_token}")
    # json_response = connect_to_endpoint(follower_url, {'pagination_token' : next_token})
    # for user in json_response['data']:
    #     print(f"{user['name']} ({user['id']})")

    following = twitter_with_cache(following_url, {'max_results': 5})
    print("\nfollowings:")
    for user in following['data']:
        print(user['name'])

    
    # print(json.dumps(json_response, indent=4, sort_keys=True))

if __name__ == "__main__":
    main()


followers:
1541427847994437637: leca, (Lecari3_)
1560377926037692417: Dwie, (Dwie42821521)
1598496097206693889: cloudtmn, (cloudtmn)
1597658439731318785: X_Hongggggg, (X_Hongggggg)
1593392589528436737: Candy Sugar Pop / Betsy, (oncestaymoa)

followings:
RBW JAPAN Official
ONEWE JAPAN
ONEUS


In [6]:
followers = twitter_with_cache(follower_url, {'max_results': 1})
print(followers)

200
{'data': [{'id': '1584836035779129346', 'name': 'Tass Hhh', 'username': 'HhhTass'}], 'meta': {'result_count': 1, 'next_token': 'FNMPNVGB6T6HGZZZ'}}
